<a href="https://colab.research.google.com/github/ProfAI/nlp00/blob/master/6%20-%20Sentiment%20Analysis/classifier_sklearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creare un classificatore con Scikit-learn
In questo notebook vedremo come creare un classificatore utilizzando scikit-learn. Lo scopo del nostro classificatore sarà quello di eseguire la **sentiment analysis** sull'IMDB movie reviews dataset, per classificare le recensioni di film come negative o positive.

## Procuriamoci il dataset
Cominciamo scaricando il dataset, esegui la cella di codice qui sotto se sei su Google Colab o se hai wget installato sul tuo computer, altrimenti puoi scaricare il dataset da [questo link](http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz).

In [1]:
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

--2019-04-16 10:02:40--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  22.3MB/s    in 5.0s    

2019-04-16 10:02:50 (16.0 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



L'archivio è compresso in formato tar.gz, esegui la cella di codice qui sotto se sei su Colab o hai l'utility tar installata sul tuo PC (solitamente è installata di base su Linux/OsX) se sei su Windows puoi usare [7-zip](https://www.7-zip.org/) (può essere che anche winrar vada bene per estrarlo).

In [0]:
!tar -xzf aclImdb_v1.tar.gz

Se abbiamo estratto il dataset adesso avremo una cartella aclImdb con dentro diversi file e altre due cartelle, una con le recensioni per l'addestramento e una con le recensioni per il test. Il dataset ci fornisce il testo di ogni recensione, ognuna in un file txt differente, organizzate in cartelle corrispondenti al sentiment positivo/negativo.
<br>
Oltre a questo ci vengono forniti dei file .feat con le recensioni già codificate utilizzando il **bag of words**, cominciamo utilizzando questi file per creare un classificatore, dopo vedremo come codificare noi stessi le recensioni.

## Metodo facile: Usiamo le features precalcolate
**ATTENZIONE** se vuoi provare ad utilizzare i file .feat non puoi utilizzare Google Colab, dato che la RAM messa a disposizione non è sufficiente, se preferisci passa direttamente al metodo 2.

Le features sono codificate in formato LIBSVM, un formato utilizzato per codificare matrici sparse. Ogni riga rappresenta una recensione, il primo numero di ogni riga corrisponde al sentiment da 1 (molto scarso) a 9 (molto buono). Gli altri valori sono in formato chiave:valore, dove la chiave è l'indice della parola all'interno del dizionario (il file imdb.vocab) e il valore è il numero di volte che la parola in questione compare all'interno della frase.
<br>
Definiamo una funzione che partendo da questo file ci permetta di ottenere la matrice che ha per ogni riga la rappresentazione bag of words di ogni recensione e il vettore con il sentiment.
<br>
Per caricare un file in formato LIBSVM possiamo usare la funzione *load_svmlight_file* di sklearn, il suo output è una tupla con all'indice 0 la matrice sparsa e all'indice 1 il vettore con i target. Utilizziamo un parametro max_features per limitare il numero di parole da tenere come features, tra quelle più frequenti.

In [0]:
import numpy as np
from sklearn.datasets import load_svmlight_file

def get_xy(file, max_features=5000, binary=True):
        
    DICTSIZE = 89522
    
    # Le parole nel dizionario sono 89522
    # se viene inserito un numero maggiore
    # come dimensione del dizionario, correggiamolo
        
    if(max_features>DICTSIZE):
        max_features = DICTSIZE
    
    dataset = load_svmlight_file(file)
    X = dataset[0].todense() # .todense() serve per convertire la matrice sparsa in densa
    
    X = np.array(X[:,:max_features])
    y = dataset[1]
    
    if(binary):
        y[y<=5] = 0 # se il punteggio è minore o uguale a 5 è una recensione negativa
        y[y>5] = 1 # se il punteggio è maggiore di 5 è una recensione positiva
        
    return (X,y)

Utilizziamo la funzione per creare gli array per l'addestramento e il test.

In [0]:
X_train, y_train = get_xy("aclImdb/train/labeledBow.feat")
X_test, y_test = get_xy("aclImdb/test/labeledBow.feat")
print(X_train.shape)
print(y_train.shape)

Standardizziamo il dataset utilizzando la classe *StandardScaler* di sklearn.

In [0]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()

X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

Creiamo il nostro modello di regressione logistica.

In [0]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(C=0.001) # C ci serve per ridurre l'overfitting 

LogisticRegression(C=0.001, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

Verifichiamo il risultato calcolando accuracy e log loss.

In [0]:
from sklearn.metrics import accuracy_score, log_loss

train_pred = lr.predict(X_train) 
train_pred_proba = lr.predict_proba(X_train)

train_accuracy = accuracy_score(y_train, train_pred)
train_loss = log_loss(y_train, train_pred_proba)

test_pred = lr.predict(X_test)
test_pred_proba = lr.predict_proba(X_test)

test_accuracy = accuracy_score(y_test, test_pred)
test_loss = log_loss(y_test, test_pred_proba)

print("Train Accuracy %.4f - Train Loss %.4f" % (train_accuracy, train_loss)) 
print("Test Accuracy %.4f - Test Loss %.4f" % (test_accuracy, test_loss)) 

Train Accuracy 0.9440 - Train Loss 0.1978
Test Accuracy 0.8776 - Test Loss 0.3126


## Metodo difficile: Estraiamo le features
Adesso proviamo a fare lo stesso, ma questa volta partendo dalle recensioni e codificandole noi stessi utilizzando il bag of words. Definiamo una funzione per leggere tutte le recensioni da tutti i files per poi ritornarle insieme al target corrispondente. Prima di ritornarle mescoliamo il dataset.

In [0]:
from os import listdir
from sklearn.utils import shuffle


def get_xy(files_path, labels=["pos","neg"]):
    
    
    label_map = {labels[0]:1, labels[1]:0}
    
    reviews = []
    y = []
    
    for label in labels:
      path = files_path+label
      for file in listdir(path):
        review_file = open(path+"/"+file)
        review = review_file.read()    
        
        reviews.append(review)
        y.append(label_map[label])
        
    # la funzione shuffle di sklearn ci permette di
    # mescolare più array allo stesso modo
    
    reviews, y = shuffle(reviews,y)
    
    return(reviews,y)

Utilizziamo la funzione per ottenere le recensioni e il target in due liste.

In [5]:
reviews_train, y_train = get_xy("aclImdb/train/")
reviews_test, y_test = get_xy("aclImdb/test/")

print("Prima recensione del set di test")
print(reviews_test[0])
print("Sentimeny: %d" % y_test[0])

Prima recensione del set di test
<br /><br />Won't be long on this movie. The first half an hour was one of the most boring i have had to face since i've started watching movies. The story didn't advanced, nothing was explained about any of the characters. It felt like a non-movie. (A lot of people had already left the audience at this point).<br /><br />A lot of the scene were totally unjustified and unexplained.<br /><br />The director should have studied film a bit more to know that each sequence, each scene, has to make the story go forward. He never did that.<br /><br />The supposedly funny moments were contrived, and only a few people laughed (people with a weird sense of humor, i guess).<br /><br />Prize of the Jury in Cannes 2002.....don't know what the jury was thinking about....probably the "politicly correct effect".<br /><br />I would have loved to love it, the disappointment was therefore even bigger.<br /><br />You have to see it to believe it. But wait for the video.
Sen

Adesso codifichiamo le recensioni, possiamo eseguire il bag of words con scikit-learn utilizzando la classe *CountVectorizer*, tramite il parametro *max_fetures* possiamo controllare il numero totale di features (e quindi di parole) da tenere, nel nostro caso teniamo soltanto le 5000 parole più frequenti.

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

bow = CountVectorizer(max_features=5000)

bow_train = bow.fit_transform(reviews_train)
bow_test = bow.transform(reviews_test)

X_train = bow_train.toarray()
X_test = bow_test.toarray()

X_train.shape

(25000, 5000)

Adesso abbiamo i nostri array per addestramento e test ! Standardizziamoli utilizzando la classe *StandardScaler* di sklearn

In [7]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()

X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


e creiamo il modello di regressione logistica, cerchiamo di ridurre un probabile 'overfitting impostando il parametro C ad un valore piccolo.

In [8]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(C=0.001)
lr.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=0.001, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

Verifichiamo il risultato calcolando l'accuracy e la log loss.

In [9]:
from sklearn.metrics import accuracy_score, log_loss

train_pred = lr.predict(X_train) 
train_pred_proba = lr.predict_proba(X_train)

train_accuracy = accuracy_score(y_train, train_pred)
train_loss = log_loss(y_train, train_pred_proba)

test_pred = lr.predict(X_test)
test_pred_proba = lr.predict_proba(X_test)

test_accuracy = accuracy_score(y_test, test_pred)
test_loss = log_loss(y_test, test_pred_proba)

print("Train Accuracy %.4f - Train Loss %.4f" % (train_accuracy, train_loss)) 
print("Test Accuracy %.4f - Test Loss %.4f" % (test_accuracy, test_loss)) 

Train Accuracy 0.9432 - Train Loss 0.1975
Test Accuracy 0.8772 - Test Loss 0.3125


Senza alcuna sorpresa, il risultato è lo stesso di prima, ma stavolta abbiamo fatto tutto noi :).

## Mettiamo all'opera il modello
Proviamo adesso ad utilizzare il modello per classificare recensioni scritte da noi, per farlo dobbiamo creare le features utilizzando lo stesso oggetto *CountVectorizer* definito sopra per poi passare le features così ottenute al metodo predict del classificatore.<br>
Ricorda che 1 equivale alla classe positiva (recensione positiva) e 0 alla classe negativa (recensione negativa).

In [12]:
review = "This is the best movie I've ever seen"
lr.predict(bow.transform([review]))

array([1])

In [13]:
review = "This is the worst movie I've ever seen"
lr.predict(bow.transform([review]))

array([0])

**NOTA BENE**
Le nuove recensioni di cui vogliamo ottenere le recensioni devono **SEMPRE** subire le stesse trasformazioni che hanno subito le recensioni del set di addestramento, per questo usiamo lo stesso oggetto di prima con il metodo *transform*. Questa è una regola generale del machine learning che si applica qualsiasi sia lo scopo predittivo del nostro modello.